# Инфраструктура для моделей машинного обучения. Практическая работа

# Цель практической работы

Потренироваться в использовании библиотек PySpark SQL и PySpark ML для предобработки данных и обучения моделей.

# Что входит в практическую работу

1. Инициализация спарк-сессии.
2. Загрузка данных.
3. Ознакомление с данными.
4. Преобразование типов столбцов.
5. Очистка данных.
6. Feature-инжиниринг.
7. Векторизация фичей.
8. Создание и обучение модели.
9. Выбор лучшей модели.
10. Обратная связь.

# Задача

Используя данные о клиентах телекоммуникационной компании, обучите модель, предсказывающую их отток.

Описание данных, с которыми вы будете работать:

* **CustomerID**: ID клиента.
* **Gender**: пол клиента.
* **SeniorCitizen**: пенсионер ли клиент (1 — да, 0 — нет).
* **Partner**: есть у клиента партнёр (жена, муж) или нет (Yes/No).
* **Dependents**: есть ли у клиента инждивенцы, например дети (Yes/No).
* **Tenure**: как много месяцев клиент оставался в компании.
* **PhoneService**: подключена ли у клиента телефонная служба (Yes/No).
* **MultipleLines**: подключено ли несколько телефонных линий (Yes, No, No phone service).
* **InternetService**: интернет-провайдер клиента (DSL, Fiber optic, No).
* **OnlineSecurity**: подключена ли у клиента услуга онлайн-безопасности (Yes, No, No internet service)
* **OnlineBackup**: подключена ли услуга резервного копирования онлайн (Yes, No, No internet service).
* **DeviceProtection**: подключена ли услуга защиты устройства (Yes, No, No internet service)
* **TechSupport**: есть ли у клиента техническая поддержка (Yes, No, No internet service).
* **StreamingTV**: подключена ли услуга потокового телевидения (Yes, No, No internet service).
* **StreamingMovies**: подключена ли услуга стримингового воспроизведения фильмов (Yes, No, No internet service).
* **Contract**: тип контракта клиента (Month-to-month, One year, Two year).
* **PaperlessBilling**: есть ли безбумажный счёт.
* **PaymentMethod**: способ оплаты услуг (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic)).
* **MonthlyCharges**: сумма, которая списывается ежемесячно.
* **TotalCharges**: сумма, списанная за всё время.
* **Churn**: ушёл ли клиент (Yes/No). Это целевая переменная, которую нужно предсказать.


# 1. Инициализация спарк-сессии

In [2]:
!pip install pyspark

In [1]:
import os

os.environ['JAVA_HOME'] = r'D:\Programs\JDK-8'
os.environ['HADOOP_HOME'] = r'D:\Programs\Hadoop'
os.environ['HADOOP_USER_NAME'] = os.getenv('USERNAME')

os.environ['PYSPARK_SUBMIT_ARGS'] = 'pyspark-shell'

from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .master("local[*]")\
    .appName('PySpark_Tutorial')\
    .getOrCreate()

# 2. Загрузка данных
Загрузите данные, сохраните их в переменную типа sparkDataframe, используя метод read.csv (не забывайте про header и delimiter).

In [3]:
sparkDataframe = spark.read.option("header",True).option("delimiter",",").csv("data.csv") 
sparkDataframe = sparkDataframe.drop(*["StreamingTV", "OnlineBackup", "PaperlessBilling", "MultipleLines"])

In [4]:
type(sparkDataframe)

pyspark.sql.dataframe.DataFrame

# 3. Ознакомление с данными
1. Выведите на экран первые несколько строк датафрейма.


In [8]:
sparkDataframe.show(3, False)

+----------+------+-------------+-------+----------+------+------------+---------------+--------------+----------------+-----------+---------------+--------------+----------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|InternetService|OnlineSecurity|DeviceProtection|TechSupport|StreamingMovies|Contract      |PaymentMethod   |MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+---------------+--------------+----------------+-----------+---------------+--------------+----------------+--------------+------------+-----+
|7590-VHVEG|Female|0            |Yes    |No        |1     |No          |DSL            |No            |No              |No         |No             |Month-to-month|Electronic check|29.85         |29.85       |No   |
|5575-GNVDE|Male  |0            |No     |No        |34    |Yes         |DSL            |Yes           |Yes             |No         |No      


2. Выведите общее количество строк датафрейма.



In [11]:
sparkDataframe.count()

7043

3. Выведите структуру (схему) датафрейма.

In [14]:
sparkDataframe.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: string (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: string (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: string (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)



# 4. Преобразование типов столбцов
Преобразуйте тип столбцов у числовых признаков (Int — если признак целочисленный, Double — если признак не целочисленный). Сохраните преобразованный датафрейм в новую переменную.

## Совет

Если вам сложно выполнить это задание, изучите дополнительные материалы: [об операторе Cast](https://sparkbyexamples.com/pyspark/pyspark-cast-column-type/), [об операторе Select](https://sparkbyexamples.com/pyspark/select-columns-from-pyspark-dataframe/).



In [17]:
from pyspark.sql.functions import expr, col

df_num = sparkDataframe

df_num = df_num.select(
    *[col(c) for c in sparkDataframe.columns if c not in ["SeniorCitizen", "tenure", "MonthlyCharges", "TotalCharges"]],
    col("SeniorCitizen").cast("Int"),
    col("tenure").cast("Int"),
    col("MonthlyCharges").cast("Double"),
    col("TotalCharges").cast("Double"),
)

In [19]:
df_num.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- Churn: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: double (nullable = true)



# 5. Очистка данных
Проверьте, есть ли в какой-либо колонке Null-значения. Для этого можно использовать your_dataframe.filter(col("colname")).isNull()).

Выведите на экран несколько строк с Null-значениями в одной из колонок.

Сохраните очищенный от строк с Null-значениями датафрейм в новую переменную. Для фильтрации этих значений можно использовать метод .isNotNull().

Колонок в датафрейме много, проверять каждую неудобно и долго. Подумайте, как упроситить эту работу, если использовать, например, перебор с циклом for.

[Примеры использования операторов isNull() и isNotNull()](https://sparkbyexamples.com/pyspark/pyspark-isnull/).


In [22]:
for i in df_num.columns:
    print(f"Пропуски для {i} - {df_num.filter(col(i).isNull()).count()}\n")

Пропуски для customerID - 0

Пропуски для gender - 0

Пропуски для Partner - 0

Пропуски для Dependents - 0

Пропуски для PhoneService - 0

Пропуски для InternetService - 0

Пропуски для OnlineSecurity - 0

Пропуски для DeviceProtection - 0

Пропуски для TechSupport - 0

Пропуски для StreamingMovies - 0

Пропуски для Contract - 0

Пропуски для PaymentMethod - 0

Пропуски для Churn - 0

Пропуски для SeniorCitizen - 0

Пропуски для tenure - 0

Пропуски для MonthlyCharges - 0

Пропуски для TotalCharges - 11



In [23]:
df_num.filter(col(i).isNull()).show(3, )

+----------+------+-------+----------+------------+---------------+-------------------+-------------------+-------------------+-------------------+--------+--------------------+-----+-------------+------+--------------+------------+
|customerID|gender|Partner|Dependents|PhoneService|InternetService|     OnlineSecurity|   DeviceProtection|        TechSupport|    StreamingMovies|Contract|       PaymentMethod|Churn|SeniorCitizen|tenure|MonthlyCharges|TotalCharges|
+----------+------+-------+----------+------------+---------------+-------------------+-------------------+-------------------+-------------------+--------+--------------------+-----+-------------+------+--------------+------------+
|4472-LVYGI|Female|    Yes|       Yes|          No|            DSL|                Yes|                Yes|                Yes|                 No|Two year|Bank transfer (au...|   No|            0|     0|         52.55|        NULL|
|3115-CZMZD|  Male|     No|       Yes|         Yes|             No|N

In [26]:
df_not_null = df_num.dropna()

# df_not_null2 = df_num.filter(col("TotalCharges").isNotNull())

In [28]:
df_not_null.count()

7032

# 6. Feature-инжиниринг
Добавьте в датафрейм одну или несколько новых фичей. Удалите колонки, которые, как вам кажется, нужно убрать из фичей. Обоснуйте свои решения.

In [31]:
from pyspark.sql.functions import when, round

# Прибыльность клиента 
df_not_null = df_not_null.withColumn("CustomerLTV",  when(col("tenure") > 0, 
         round(col("TotalCharges") / col("tenure") * 12, 2) 
        ).otherwise(0)).drop("customerID")

In [33]:
df_not_null.show()

+------+-------+----------+------------+---------------+-------------------+-------------------+-------------------+-------------------+--------------+--------------------+-----+-------------+------+--------------+------------+-----------+
|gender|Partner|Dependents|PhoneService|InternetService|     OnlineSecurity|   DeviceProtection|        TechSupport|    StreamingMovies|      Contract|       PaymentMethod|Churn|SeniorCitizen|tenure|MonthlyCharges|TotalCharges|CustomerLTV|
+------+-------+----------+------------+---------------+-------------------+-------------------+-------------------+-------------------+--------------+--------------------+-----+-------------+------+--------------+------------+-----------+
|Female|    Yes|        No|          No|            DSL|                 No|                 No|                 No|                 No|Month-to-month|    Electronic check|   No|            0|     1|         29.85|       29.85|      358.2|
|  Male|     No|        No|         Yes|

# 7. Векторизация фичей
Подготовьте данные к обучению:





1. Преобразуйте текстовые колонки в числа, используя StringIndexer.
Удалите столбцы со старыми (непреобразованными) признаками. Выведите на экран структуру получившегося датафрейма. Не забывайте о столбце Churn. Хоть он и выступает в задаче как таргет, он имеет текстовый тип, поэтому тоже должен быть закодирован числовыми значениями.

Чтобы использовать StringIndexer для всех категориальных признаков сразу, а не для каждого отдельно, можно применить сущность pipeline.

**Пример кода:**

##### #Задаём список текстовых колонок:
text_columns = ["text_col_1", "text_col_2", "text_col_3"]

##### #Задаём список StringIndexer'ов — сущностей, каждая из которых будет кодировать одну текстовую колонку числами. Имена преобразованных колонок будут заканчиваться на _index:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index",).fit(<ваш датасет>) for column in text_columns]

##### #Создаём Pipeline из StringIndexer'ов:
pipeline = Pipeline(stages=indexers)

##### #Скармливаем нашему pipeline датафрейм, удаляя старые колонки:
new_dataframe = pipeline.fit(<ваш датасет>).transform(<ваш датасет>).drop(*text_columns)


In [37]:
from pyspark import mllib
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

df_not_proc = df_not_null

#список колонок с текстовым типом
text_cols = ["gender", "Partner", "Dependents", "PhoneService", "InternetService", \
                "OnlineSecurity" , "DeviceProtection", "TechSupport", \
                "StreamingMovies", "Contract", "PaymentMethod", "Churn"]

indexers = [StringIndexer(inputCol=column, outputCol=column+"_idx",).fit(df_not_proc) for column in text_cols]

pipeline = Pipeline(stages=indexers)

df_proc = pipeline.fit(df_not_proc).transform(df_not_proc).drop(*text_cols)

In [38]:
df_proc.show()

+-------------+------+--------------+------------+-----------+----------+-----------+--------------+----------------+-------------------+------------------+--------------------+---------------+-------------------+------------+-----------------+---------+
|SeniorCitizen|tenure|MonthlyCharges|TotalCharges|CustomerLTV|gender_idx|Partner_idx|Dependents_idx|PhoneService_idx|InternetService_idx|OnlineSecurity_idx|DeviceProtection_idx|TechSupport_idx|StreamingMovies_idx|Contract_idx|PaymentMethod_idx|Churn_idx|
+-------------+------+--------------+------------+-----------+----------+-----------+--------------+----------------+-------------------+------------------+--------------------+---------------+-------------------+------------+-----------------+---------+
|            0|     1|         29.85|       29.85|      358.2|       1.0|        1.0|           0.0|             1.0|                1.0|               0.0|                 0.0|            0.0|                0.0|         0.0|         

2. Векторизуйте категориальные признаки, используя OneHotEncoder.
Удалите столбцы со старыми (непреобразованными) признаками.
Выведите на экран структуру получившегося после преобразований датафрейма.


In [42]:
from pyspark.ml.feature import OneHotEncoder

#список категориальных колонок
features_inp  = ["gender", "Partner", "Dependents", "PhoneService", "InternetService", \
                "OnlineSecurity", "DeviceProtection", "TechSupport",  \
                "StreamingMovies", "Contract",  "PaymentMethod"]


encoders = [OneHotEncoder(inputCol=column+"_idx", outputCol=column+"_vec")
            for column in features_inp]

encoder_pipeline = Pipeline(stages=encoders)

df_encoded = encoder_pipeline.fit(df_proc).transform(df_proc)

index_cols_to_drop = [col+"_idx" for col in features_inp]
df_encoded = df_encoded.drop(*index_cols_to_drop)

# Переименовываем
df_encoded = df_encoded.withColumnRenamed("Churn_idx", "target")

In [43]:
df_encoded.show()

+-------------+------+--------------+------------+-----------+------+-------------+-------------+--------------+----------------+-------------------+------------------+--------------------+---------------+-------------------+-------------+-----------------+
|SeniorCitizen|tenure|MonthlyCharges|TotalCharges|CustomerLTV|target|   gender_vec|  Partner_vec|Dependents_vec|PhoneService_vec|InternetService_vec|OnlineSecurity_vec|DeviceProtection_vec|TechSupport_vec|StreamingMovies_vec| Contract_vec|PaymentMethod_vec|
+-------------+------+--------------+------------+-----------+------+-------------+-------------+--------------+----------------+-------------------+------------------+--------------------+---------------+-------------------+-------------+-----------------+
|            0|     1|         29.85|       29.85|      358.2|   0.0|    (1,[],[])|    (1,[],[])| (1,[0],[1.0])|       (1,[],[])|      (2,[1],[1.0])|     (2,[0],[1.0])|       (2,[0],[1.0])|  (2,[0],[1.0])|      (2,[0],[1.0])|(

3. Объедините колонки фичей в один вектор, используя VectorAssembler.
Удалите столбцы со старыми (непреобразованными) признаками.
Выведите на экран первые несколько строк и структуру получившегося датафрейма.

In [47]:
from pyspark.ml.feature import VectorAssembler

numeric_cols = ["SeniorCitizen", "tenure", "MonthlyCharges", "TotalCharges", "CustomerLTV"]

vector_cols = [col+"_vec" for col in features_inp]

assembler = VectorAssembler(
    inputCols=numeric_cols + vector_cols,
    outputCol="features"
)

df_final = assembler.transform(df_encoded)

cols_to_drop = numeric_cols + vector_cols
df_final = df_final.drop(*cols_to_drop)

In [49]:
df_final.show(2, False)

+------+-----------------------------------------------------------------------------------------------------------+
|target|features                                                                                                   |
+------+-----------------------------------------------------------------------------------------------------------+
|0.0   |(24,[1,2,3,4,7,10,11,13,15,17,19,21],[1.0,29.85,29.85,358.2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])              |
|0.0   |(24,[1,2,3,4,5,6,7,8,10,12,14,15,17,22],[34.0,56.95,1889.5,666.88,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
+------+-----------------------------------------------------------------------------------------------------------+
only showing top 2 rows



# 8. Создание и обучение модели

1. Создайте модель — логистическую регрессию (используя LogisticRegression). В качестве параметров класса LogisticRegression укажите колонку фичей (параметр featuresCol), колонку-таргет (параметр labelCol) из датафрейма и имя колонки, в которую будут записываться предсказания (параметр predictionCol).

In [53]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(
    featuresCol="features",  # колонка с фичами
    labelCol="target",        # колонка с таргетом
    predictionCol="prediction",  # колонка для предсказаний
    maxIter=100              # максимальное количество итераций
)

2. Разделите датафрейм на обучающую и тестовую выборку.

In [56]:
train_data, test_data = df_final.randomSplit([0.7, 0.3], seed=42)

3. Создайте объекты — сетки гиперпараметров для каждой модели, используя ParamGridBuilder. Так же, как и в ноутбуке из последнего видео, в сетку гиперпараметров можно добавить значения параметров regParam и elasticNetParam.

Вы можете ознакомиться [с документацией объекта LogisticRegression в PySpark](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.LogisticRegression.html) и добавить в сетку больше параметров.


In [80]:
from pyspark.ml.tuning import ParamGridBuilder

paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.1]) \
    .addGrid(lr.maxIter, [100]) \
    .build()

4. Создайте объект evaluator, который будет отвечать за метрику качества при обучении. Для этого используйте класс BinaryClassificationEvaluator со следующими параметрами: rawPredictionCol — колонка с предсказаниями, labelCol — колонка с таргетом.

У вас, возможно, возник вопрос, какую метрику качества берёт по умолчанию BinaryClassificationEvaluator. По умолчанию BinaryClassificationEvaluator будет рассчитывать areaUnderROC. Это метрика оценки площади под кривой ROC (Receiver Operating Characteristic), которая служит графической интерпретацией производительности модели. Эта метрика качества находится в пределах от 0 до 1. Чем выше метрика, тем более качественные предсказания делает модель.

In [62]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(
    rawPredictionCol="rawPrediction",  # колонка с raw предсказаниями
    labelCol="target",                  # колонка с таргетом
    metricName="areaUnderROC"          # метрика качества
)

5. Создайте объект CrossValidator, в качестве параметров укажите уже созданные вами модель, сетку гиперпараметров и evaluator.

In [82]:
from pyspark.ml.tuning import CrossValidator

cv = CrossValidator(
    estimator=lr,                      # модель для обучения
    estimatorParamMaps=paramGrid,      # сетка гиперпараметров
    evaluator=evaluator,               # evaluator для оценки
    numFolds=3,                        # количество фолдов для кросс-валидации
    seed=42,                           # seed для воспроизводимости
    parallelism=2                      # параллельность для ускорения
)

6. Запустите обучение модели на тренировочной выборке. Сохраните обученную модель в новую переменную.

In [84]:
cv_model = cv.fit(train_data)

# 9. Выбор лучшей модели

1. Выберите лучшую модель, сохраните её в отдельную переменную, отобразите её параметры.

Вывод параметров модели в PySpark можно сделать, используя метод extractParamMap().

In [86]:
best_model = cv_model.bestModel

params = best_model.extractParamMap()
for param, value in params.items():
    print(f"  {param.name}: {value}")

  aggregationDepth: 2
  elasticNetParam: 0.1
  family: auto
  featuresCol: features
  fitIntercept: True
  labelCol: target
  maxBlockSizeInMB: 0.0
  maxIter: 100
  predictionCol: prediction
  probabilityCol: probability
  rawPredictionCol: rawPrediction
  regParam: 0.01
  standardization: True
  threshold: 0.5
  tol: 1e-06


2. Запустите лучшую модель в режиме предсказания на тестовой выборке. Сохраните предсказания в отдельную переменную. Выведите первые несколько строк датафрейма с предсказаниями на экран.

Запуск модели в режиме предсказания выполняется при помощи метода .transform(<тестовая выборка>).

In [88]:
predictions = best_model.transform(test_data)

test_predictions = predictions

In [92]:
test_predictions.select(
    "target",                      # Истинное значение
    "prediction",                 # Предсказанное значение (0 или 1)
    "probability",                # Вероятность принадлежности к классу 1
    "rawPrediction"               # Raw prediction (логиты)
).show(10, truncate=False)

+------+----------+-----------------------------------------+------------------------------------------+
|target|prediction|probability                              |rawPrediction                             |
+------+----------+-----------------------------------------+------------------------------------------+
|0.0   |0.0       |[0.7122698786403454,0.2877301213596546]  |[0.9064339204365137,-0.9064339204365137]  |
|0.0   |0.0       |[0.9882736902507171,0.011726309749282904]|[4.434124660020761,-4.434124660020761]    |
|0.0   |0.0       |[0.9881484024595553,0.011851597540444736]|[4.423370219352572,-4.423370219352572]    |
|0.0   |0.0       |[0.9863582830449906,0.013641716955009375]|[4.280887137867351,-4.280887137867351]    |
|0.0   |0.0       |[0.9631277912926047,0.03687220870739527] |[3.2627279885082663,-3.2627279885082663]  |
|0.0   |0.0       |[0.5459562344792261,0.45404376552077386] |[0.18434522268006015,-0.18434522268006015]|
|0.0   |1.0       |[0.3775299884019014,0.62247001159809

3. Получите метрику качества модели. Для этого примените к объекту evaluator метод .evaluate(<ваш датафрейм с предсказаниями>).



In [94]:
test_auc = evaluator.evaluate(test_predictions)

test_auc

0.836403569816551